In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from subprocess import check_output

In [3]:
import subprocess
import sys

site = sys.argv[1]
ping_count = 4
process = subprocess.Popen(['ping', site, '-c', str(ping_count)],
                           stdout=subprocess.PIPE,
                           stderr=subprocess.STDOUT)
returncode = process.wait()
print('ping returned {0}'.format(returncode))
print(process.stdout.read())

ping returned 77
b'ping: -f flag: Operation not permitted\n'


In [11]:
diff=pd.read_csv('diffsydiw.csv')
print(diff.head())

   syd    did  wei
0  1.0  163.0  2.0
1  1.0  164.0  2.0
2  1.0  165.0  1.0
3  1.0  187.0  2.0
4  1.0  306.0  2.0


In [7]:
sym=pd.read_csv('sym_t.csv')
dia=pd.read_csv('dia_t.csv')
print(sym.head())

   syd                      symptom
0    1         Upper abdominal pain
1    2         Lower abdominal pain
2    3  Abscess (Collection of pus)
3    4                Alcohol abuse
4    5        Anxiety (Nervousness)


In [8]:
print(dia.head())

   did                                           diagnose
0    1  Abdominal aortic aneurysm(enlarged major bloo...
1    2                                 Abdominal swelling
2    3                                   Abdominal trauma
3    4                               Abrasions (scrapes)
4    5  ACE inhibitor induced coughblood pressure med...


In [9]:
sd_diff=diff.merge(sym, left_on='syd', right_on='syd')

In [10]:
print(sd_diff.head())

   syd    did  wei               symptom
0  1.0  163.0  2.0  Upper abdominal pain
1  1.0  164.0  2.0  Upper abdominal pain
2  1.0  165.0  1.0  Upper abdominal pain
3  1.0  187.0  2.0  Upper abdominal pain
4  1.0  306.0  2.0  Upper abdominal pain


In [16]:
sd_diff=sd_diff.merge(dia, left_on='did', right_on='did')
print(sd_diff)

        syd     did  wei               symptom  \
0       1.0   163.0  2.0  Upper abdominal pain   
1       2.0   163.0  1.0  Lower abdominal pain   
2       7.0   163.0  1.0     Back ache or pain   
3      29.0   163.0  2.0                 Fever   
4      32.0   163.0  1.0            Flank pain   
...     ...     ...  ...                   ...   
5563  271.0  1196.0  2.0       Pulling at ears   
5564  272.0  1170.0  2.0            Skin bumps   
5565  272.0  1184.0  0.0            Skin bumps   
5566  272.0  1186.0  0.0            Skin bumps   
5567  272.0   899.0  0.0            Skin bumps   

                                         diagnose_x  \
0     Cholecystitisinflammation of the gallbladder   
1     Cholecystitisinflammation of the gallbladder   
2     Cholecystitisinflammation of the gallbladder   
3     Cholecystitisinflammation of the gallbladder   
4     Cholecystitisinflammation of the gallbladder   
...                                             ...   
5563     Eusta

In [14]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [15]:
from scipy.sparse import coo_matrix, csr_matrix

In [18]:
def read_data(filename):
    """ Reads in the last.fm dataset, and returns a tuple of a pandas dataframe
    and a sparse matrix of song/user/playcount """
    # read in triples of user/song/playcount from the input dataset
    data = pd.read_csv(filename,
                             usecols=[0,1,2],        #[36, 11, 10] vrk_pat_primkey,prd_atc_primkey,vdp_aantal
                             names=['user', 'song','plays'],skiprows=1) #[:1000000]   # user = patient, or prescriptionnr song=atc
                                #skiprows= skip N rows from start of the file
    data=data.dropna(axis=0, how='any')  #drop nan
    data['plays']=data['plays']+1
    print(data.head())
    # map each song and user to a unique numeric value
    data['user'] = data['user'].astype("category")
    data['song'] = data['song'].astype("category")

    # create a sparse matrix of all the users/plays
    plays = coo_matrix((data['plays'].astype(float),
                       (data['song'].cat.codes.copy(), # cat codes return series of codes as well as index
                        data['user'].cat.codes.copy())))
    
    return data, plays,data.groupby(['song']).plays.sum(),data['user'].cat.codes.copy()

data,matrix,songsd,user=read_data('diffsydiw.csv')
data.head(10)

#user=symptom
#sond=diagnose

   user   song  plays
0   1.0  163.0    3.0
1   1.0  164.0    3.0
2   1.0  165.0    2.0
3   1.0  187.0    3.0
4   1.0  306.0    3.0


,user,song,plays
0,1.0,163.0,3.0
1,1.0,164.0,3.0
2,1.0,165.0,2.0
3,1.0,187.0,3.0
4,1.0,306.0,3.0
5,1.0,307.0,2.0
6,1.0,308.0,3.0
7,1.0,309.0,3.0
8,1.0,354.0,2.0
9,1.0,401.0,2.0


# Normalize

In [19]:
from sklearn.preprocessing import normalize


def cosine(plays):
    normalized = normalize(plays)# normalize the data so that it wont distort the difference between the value
    return normalized.dot(normalized.T)


def bhattacharya(plays):
    plays.data = np.sqrt(plays.data) #RMS value
    return cosine(plays)


def ochiai(plays):
    plays = csr_matrix(plays) # compressed sparse row matrix
    plays.data = np.ones(len(plays.data))
    return cosine(plays)


def bm25_weight(data, K1=1.2, B=0.8):
    """ Weighs each row of the matrix data by BM25 weighting """
    # calculate idf per term (user)
    N = float(data.shape[0])
    idf = np.log(N / (1 + np.bincount(data.col)))#bincount=it counts the occurrence of each element

    # calculate length_norm per document (artist)
    row_sums = np.squeeze(np.asarray(data.sum(1)))
    average_length = row_sums.sum() / N
    length_norm = (1.0 - B) + B * row_sums / average_length

    # weight matrix rows by bm25
    ret = coo_matrix(data)
    ret.data = ret.data * (K1 + 1.0) / (K1 * length_norm[ret.row] + ret.data) * idf[ret.col]
    return ret


def bm25(plays):
    plays = bm25_weight(plays)
    return plays.dot(plays.T)

def get_largest(row, N=10):
    if N >= row.nnz:
        best = zip(row.data, row.indices)
    else:
        ind = np.argpartition(row.data, -N)[-N:]
        best = zip(row.data[ind], row.indices[ind])
    return sorted(best, reverse=True)

def calculate_similar_artists(similarity, artists, artistid):
    neighbours = similarity[artistid]
    top = get_largest(neighbours)
    return [(artists[other], score, i) for i, (score, other) in enumerate(top)]


#songsd = dict(enumerate(data['song'].cat.categories))
user_count = data.groupby('user').size()
#to_generate = sorted(list(songsd), key=lambda x: -user_count[x])

similarity = bm25_weight(matrix)

In [20]:
print(sym)
sym[sym['syd'].isin(list(songsd.index))]

     syd                          symptom
0      1             Upper abdominal pain
1      2             Lower abdominal pain
2      3      Abscess (Collection of pus)
3      4                    Alcohol abuse
4      5            Anxiety (Nervousness)
..   ...                              ...
267  295                 Nipple discharge
268  301  Shoulder stiffness or tightness
269  303       Arm stiffness or tightness
270  304              High blood pressure
271  306                 High blood sugar

[272 rows x 2 columns]


,syd,symptom
0,1,Upper abdominal pain
1,2,Lower abdominal pain
2,3,Abscess (Collection of pus)
3,4,Alcohol abuse
4,5,Anxiety (Nervousness)
...,...,...
266,294,Confusion and headache
267,295,Nipple discharge
269,303,Arm stiffness or tightness
270,304,High blood pressure


In [21]:
from scipy.sparse.linalg import svds

Ur, Si, VTr = svds(bm25_weight(coo_matrix(matrix)), k=100)
print(Ur.shape, Si.shape, VTr.shape,user.shape,matrix.shape,data.shape,songsd.shape,user_count.shape)
VTr=pd.DataFrame(VTr)

(1145, 100) (100,) (100, 272) (5371,) (1145, 272) (5371, 3) (1145,) (272,)


In [22]:
from sklearn.metrics.pairwise import cosine_similarity
Sddf=pd.DataFrame(cosine_similarity(Ur,VTr.T),columns=user_count.index,index=list(songsd.index)) 
Sddf.to_csv('Sddf.csv')

Sydi=pd.DataFrame(cosine_similarity(Ur,VTr.T))

In [23]:
booknr=41 #symptoom4
print('Symptom',sym[sym['syd']==booknr])
print('top 7 related disease probability') #,Sddf[booknr].sort_values(ascending=False))
print()
lijst= Sddf[booknr].sort_values(ascending=False).index 
for xi in lijst[:7]:
    print(xi,dia[dia['did']==xi].diagnose.values)

Symptom     syd   symptom
40   41  Headache
top 7 related disease probability

596.0 ['Posterior inferior cerebellar artery syndrome\x0bstroke syndrome']
173.0 ['Cluster headache\x0bone-sided headache behind the eye']
505.0 ['Nasal fracture\x0bbroken nose']
1283.0 ['Farsightedness\x0bhyperopia/hypermetropia']
782.0 ['Trigeminal neuralgia\x0bfacial nerve pain']
750.0 ['Temporomandibular joint syndrome\x0bTMJ, jaw joint pain']
1389.0 ['Arnold Chiari malformation']


In [24]:
lijst=list(lijst[:3])
vectori=[0 for x in range(0,len(user_count))]
for xi in lijst:
    #ii=songsd.index(xi)
    vectori+=Sddf.loc[xi]
vect=pd.DataFrame(vectori)
vect.columns=['para']
vect=vect[vect>1].dropna(axis=0)
print('other symptoms',vect/3)
print(sym[sym.syd.isin(list(vect.index))])


other symptoms            para
user           
27.0   0.551396
41.0   0.793294
65.0   0.386402
236.0  0.372055
     syd           symptom
26    27       Facial pain
40    41          Headache
64    65      Nasal injury
223  236  Nasal congestion
